# PPCS Application Walkthrough

Production Planning and Control System

## Initialize

In [9]:
import pandas as pd

from ppcs.storage import DatabaseManager
from ppcs.utils import init_logger

APP_NAME = "ppcs"

logger = init_logger(APP_NAME)
dbm = DatabaseManager()

## Data exploration - Job Input Parameters

Here we have a list of jobs, planned input dates and targetted dates to output

In [2]:
df_input_jobs = dbm.get_df_input_jobs()
df_input_jobs

,lo_id,planned_input_timestamp,planned_output_timestamp,planned_tat_days
0,j000000001,2024-06-25 08:00:00,2024-09-23 08:00:00,90 days
1,j000000002,2024-06-11 08:00:00,2024-08-20 08:00:00,70 days
2,j000000003,2024-06-04 08:00:00,2024-09-02 08:00:00,90 days
3,j000000004,2024-06-13 08:00:00,2024-08-12 08:00:00,60 days
4,j000000005,2024-06-13 08:00:00,2024-08-12 08:00:00,60 days
5,j000000006,2024-07-16 08:00:00,2024-09-24 08:00:00,70 days
6,j000000007,2024-07-04 08:00:00,2024-10-02 08:00:00,90 days
7,j000000008,2024-07-18 08:00:00,2024-09-16 08:00:00,60 days
8,j000000009,2024-07-11 08:00:00,2024-10-09 08:00:00,90 days
9,j000000010,2024-07-25 08:00:00,2024-10-23 08:00:00,90 days


Workscopes are the planned tasks/processes/repairs that required to be perform on a given engine job.

In this table, you will find the `lo_id`,  `process_code_masked`, `manhour_to_be_planned` to perform the engine job.

- `lo_id` means leading order ID, or Job ID.
- `process_code_masked` means the name of the process, or repair task that needs to be performed. 
- `manhour_to_be_planned` means the planned manhours needed to get the task done.

In [3]:
df_input_workscope = dbm.get_df_input_workscope()
df_input_workscope

,Unnamed: 0,job_process_index,lo_id,process_code_masked,manhour_to_be_planned
0,0,0,1,lpt_major_module-gate010-CUCEAA-binsp,29.501293
1,1,1,1,lpt_major_module-gate010-CUCEAA-clean,82.531458
2,2,2,1,lpt_major_module-gate010-CUCEAA-ndtest,19.103398
3,3,3,1,accessory_drive_module-gate010-CUCKAA-binsp,23.396815
4,4,4,1,accessory_drive_module-gate030-CUCKAA-sbassm,128.991356
...,...,...,...,...,...
4066,4066,269,15,fan_major_module-gate010-CUAWAA-utstrip,15.822293
4067,4067,270,15,lpt_major_module-gate010-CUCCAA-utstrip,66.095223
4068,4068,271,15,core_major_module-gate010-CUCAAA-utstrip,7.316237
4069,4069,272,15,lpt_major_module-gate010-CUCDAA-utstrip,26.873341


## Data exploration - Job Input Parameters

Here we have a database to record the capability of our repair shop.

### Manpower

A table of employees and skillset that they are capable of.

Each capable process is separated by `;` in the `process_code` column.

In [4]:
df_manpower = dbm.get_df_manpower()
df_manpower

,emp_id,process_code,skillset_count,designation,workshift
0,st00000023,core_major_module-gate010-CUCBAA-clean;core_ma...,17,Service Engineer,NORM
1,st00000050,others-gate022-CTAAAA-eiclean;others-gate022-C...,15,Service Engineer,NORM
2,st00000011,lpt_major_module-gate010-CUCDAA-ndtest;engine_...,15,Senior Technician I,NORM
3,st00000092,engine_major-gate999-CUAAAD-suppt;fan_major_mo...,12,Senior Workshop Engineer,NORM
4,st00000052,others-gate022-CVAAAA-eiclean;others-gate022-D...,10,Senior Technician I,NORM
...,...,...,...,...,...
96,st00000112,core_major_module-gate021-CUCAAA-rprplsm;lpt_m...,2,Senior Technician I,NS
97,st00000091,core_major_module-gate021-CUBPAA-rprpntg;engin...,2,Senior Technician I,NS
98,st00000085,core_major_module-gate021-CUCBAA-rprmach;core_...,2,Senior Technician II,NS
99,st00000106,others-gate021-CSAAAA-rprpntg;core_major_modul...,2,Senior Technician II,NS


In [5]:
df_shift_table = dbm.get_shift_table()
df_shift_table

,shift_code,shift_description,shift_start_hour,shift_end_hour,shift_days
0,N,normal shift,0800H,1730H,Monday; Tuesday; Wednesday; Thursday; Friday
1,DS,day shift,0700H,1630H,Monday; Tuesday; Wednesday; Thursday; Friday
2,NS,night shift,1600H,0130H,Monday; Tuesday; Wednesday; Thursday; Friday


A table of all possible repair processes, and the machines_id that are available in the shop.

In [7]:
df_shopfloor = dbm.get_df_shopfloor()
df_shopfloor.head(5)

,process_code,hour_per_man_mean,appeared_in_n_jobs,manhour_required_mean,max_no_of_man,hour_per_man_std,manhour_required_std,machine_ids
0,accessory_drive_module-gate010-CUCKAA-binsp,9.016796,14,9.157282,2,8.076928,7.961147,BSBAY01; BSBAY02; BSBAY03; BSBAY04
1,accessory_drive_module-gate010-CUCKAA-clean,32.700833,4,39.534722,2,37.469202,38.639022,CLNTK01; CLNTK02; CLNTK03; CLNTK04; CLNTK05
2,accessory_drive_module-gate010-CUCKAA-grstrip,15.097315,3,18.113241,2,11.620146,10.372095,EACBAY01; EACBAY02; EACBAY03; EACBAY04; EACBAY...
3,accessory_drive_module-gate010-CUCKAA-inspt,10.051888,7,39.997302,6,5.962089,39.342920,INSPBAY01; INSPBAY02; INSPBAY03; INSPBAY04; IN...
4,accessory_drive_module-gate010-CUCKAA-ndtest,11.132926,5,31.971000,4,3.159584,14.777426,DARKRM01; DARKRM02; DARKRM03; DARKRM04; DARKRM...


## Optimizer

Here, we attempt to create an optimizer tool that is able to take the jobs_input, then allocate the resources `manpower` and `machines`, generating a gantt chart timeline as an output

In [8]:
class Optimizer:
    def __init__(self, df_job_workscope: pd.DataFrame, constraints: dict) -> None:
        pass